
# 🌟 **Career Agent (LLM-Powered Job Search Assistant)**

This notebook walks through a complete mini-project where an agent helps candidates with **job search assistance** using Gemini, LangChain, LangGraph, and Tavily Search.

It parses a resume → generates a job query → fetches job postings → matches them → and finally writes personalized cover letters.

<br>

## 🔍 **What You Will Learn (Simple & Project-Focused)**

* How to build a complete workflow using LangGraph
* How to parse resumes using structured LLM output
* How to perform job searches using an external API
* How to match candidate profiles with real job listings
* How to generate customized cover letters automatically

<br>

## 🧠 **Prerequisites**

* Basic Python
* Google API Key & Tavily API Key
* A resume text input


# Career Agent: LLM-Powered Job Search Assistant

One-liner explaining the purpose and outcome of the notebook: This notebook uses an LLM and job search tool to help candidates find relevant jobs and write personalized cover letters.

## What this does

*   Parses resume text into a structured profile.
*   Generates a targeted job search query and fetches postings.
*   Scores relevance and selects top matches with reasons.
*   Drafts personalized cover letters for each selected job.

## Quick start

1.  Run “Install” → “Keys” → “Config” → “Run Pipeline” in order.
2.  Replace the sample resume in the “Input resume” cell before running the pipeline.

## Requirements

*   Python 3.x runtime (Colab).
*   Google Generative AI key and Tavily key stored as secrets or environment variables.

### Installing Required Packages
We install the core dependencies needed for our Career Agent, including LangChain, Google GenAI, and supporting libraries.


In [ ]:
!pip install -qU langchain-google-genai grandalf langchain langchain_community langchain_core langgraph

### Setting Up API Keys
Here we securely load API keys from Colab’s `userdata`


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

### Initializing the Model Configuration
This section sets up the connection to the Google Generative AI model using LangChain integrations.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',google_api_key=GOOGLE_API_KEY)
response = llm.invoke("What is Your knowledge Cutoff!")
print(response.content)

My knowledge cutoff is **June 2024**. Therefore, I cannot provide you with any information about events or 
developments that have occurred since that time.

### Understanding the Project Logic
Below we define the core structure and flow of the Career Agent — how data moves between components and what the model processes.


### Project explanation

This project uses a Langgraph state machine to orchestrate the different steps of the job search assistance process. The state graph manages the flow of information and execution between the different nodes, each responsible for a specific task.

#### Architecture

The flow of the application is as follows:



```
# Entry → Profile Node → Job Search Node → Matcher Node → Cover Letter Node → END
```



*   **Entry**: The process starts with the user providing their resume.
*   **Profile Node**: The resume text is parsed to extract key profile information.
*   **Job Search Node**: A search query is generated based on the profile, and job postings are fetched using the Tavily API.
*   **Matcher Node**: The fetched job postings are evaluated against the candidate's profile to determine relevance and select the top matches.
*   **Cover Letter Node**: Personalized cover letters are drafted for the top job matches.
*   **END**: The process concludes with the generated cover letters.

Orchestration is handled by a state graph where outputs from each node are stored in variables for display and subsequent node processing.

In [ ]:
from typing import TypedDict, Annotated
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph,END,add_messages
from langchain_core.tools import tool

### Defining Agent State
This class defines the data structure (`MyState`) used to manage input, output, and intermediate states within the agent workflow.


In [ ]:
class MyState(TypedDict):
  messages: Annotated[list,add_messages]

tavily = TavilySearchResults(tavily_api_key=TAVILY_API_KEY,max_results=10)

### Setting Up Output Parsing
We use LangChain’s `StructuredOutputParser` and `ResponseSchema` to format the model responses into readable and structured outputs.


In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

In [ ]:
import json

# =========== Tools =============

@tool
def parse_resume(text:str) -> str:
  """
    Parse a resume and return the candidate's profile as a structured string.

    Fields extracted: name, current_position, years_experience, skills, education,
    certifications, location.

    Args:
        text (str): Resume text.

    Returns:
        str: Structured profile as a JSON-like string.
    """

  # Define the fields you want to extract
  response_schemas = [
      ResponseSchema(name="name", description="Full name of the candidate"),
      ResponseSchema(name="current_position", description="Current job title or position"),
      ResponseSchema(name="years_experience", description="Number of years of experience"),
      ResponseSchema(name="skills", description="List of key skills"),
      ResponseSchema(name="education", description="List of educational qualifications"),
      ResponseSchema(name="certifications", description="List of certifications"),
      ResponseSchema(name="location", description="City and country of the candidate"),
  ]
  parser = StructuredOutputParser.from_response_schemas(response_schemas)
  format_instructions = parser.get_format_instructions()
  template="""
      You are a professional resume parser.
      Extract the candidate's profile from the resume text below.
      {format_instructions}

      Resume Text:
      {resume_text}
  """
  prompt = PromptTemplate(
      template=template,
      input_variables=["resume_text"],
      partial_variables={"format_instructions": format_instructions}
  )
  parsing_chain = prompt | llm | parser
  response = json.dumps(parsing_chain.invoke({"resume_text":text}))
  return response

@tool
def search_jobs_tavily(query:str)->str:
  """
    Search for jobs on Tavily based on a query string.

    Args:
        query (str): The search query or keywords for the job search.

    Returns:
        str: A string representation of the search results.

    Example:
        search_jobs_tavily("Python Developer")
        # Returns a string containing job listings related to Python Developer
  """
  results = tavily.invoke(query)
  print(f"✨ Tavily Search Results: {results}")
  print(f"ℹ️ Type of results: {type(results)}")
  results = json.dumps(results,indent=2)
  print(f"ℹ️ Type of results after JSON dump: {type(results)}")
  return results

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# =========== Nodes =============

def profile_node(state:MyState):
  resume = state["messages"][-1].content
  profile = parse_resume(resume)
  state["messages"].append(AIMessage(content=profile))
  return state

def job_search_node(state:MyState):
  parser=StrOutputParser()
  profile = state["messages"][-1].content
  from langchain.prompts import PromptTemplate
  template="""
    You are a Job Search Query Generator. Your task is to take a candidate's profile and convert it
    into a concise, precise search query suitable for the Tavily job search API.

    Input (JSON-like string with candidate profile):
    {profile_json}

    Instructions:
    1. Extract key details: skills, current_position, desired_roles (if available), location, and other preferences.
    2. Combine them into a single natural-language query for searching jobs.
    3. Prioritize skills and desired roles first, then location, then other preferences.
    4. Output only the search query string; do not include explanations, labels, or extra text.

    Example Input:
    {{
        "name": "Alice",
        "current_position": "Software Engineer",
        "years_experience": 3,
        "skills": ["Python", "Django", "React"],
        "education": ["B.Tech in CS"],
        "certifications": ["AWS Certified Developer"],
        "location": "Bangalore",
        "desired_roles": ["Full Stack Developer"],
        "other_preferences": "Remote work preferred"
    }}

    Example Output:
    "Full Stack Developer jobs in Bangalore with Python, Django, React skills, remote work preferred"

    Now, generate a search query for the following profile:
    {profile_json}
"""

  # Define the prompt template
  job_query_template = PromptTemplate(
      template=template,
      input_variables=["profile_json"]
  )
  chain = job_query_template | llm | parser
  query = chain.invoke({"profile_json":profile})
  results = search_jobs_tavily(query)
  state["messages"] = [AIMessage(content=results)] # Another valid way to append messages (thanks to add_messages field)
  return state

def matcher_node(state:MyState):
  parser = StrOutputParser()
  profile = state["messages"][-2].content
  results = state["messages"][-1].content
  template="""
    You are a Job-Profile Matcher. Your task is to evaluate how well a list of job postings matches a candidate's profile
    and select the **top 3–5 most relevant jobs** based on their match score and significance.

    Candidate Profile (JSON string):
    {profile_json}

    Job Results (JSON array string, each job has "title", "company"/"content", "url"):
    {job_results}

    Instructions:
    1. For each job, compare it with the candidate profile considering:
      - skills
      - current_position / desired_roles
      - location
      - experience
      - other preferences
    2. Assign a match score from 0 to 10 (10 = perfect match).
    3. Give a short reason for the score.
    4. After scoring all jobs, **select only the top 3–5 jobs** based on highest match score.
    5. Output as a JSON array where each element is:
      {{
          "job_title": "...",
          "company": "...",
          "match_score": ...,
          "reason": "...",
          "url": "..."
      }}

    Example Output:
    [
      {{
        "job_title": "Full Stack Developer",
        "company": "TechCorp",
        "match_score": 9,
        "reason": "Skills match well, location is preferred, role matches desired role.",
        "url": "https://tavily.com/jobs/12345"
      }},
      {{
        "job_title": "Backend Engineer",
        "company": "CodeLabs",
        "match_score": 8,
        "reason": "Good skills match, location acceptable, role slightly different.",
        "url": "https://tavily.com/jobs/67890"
      }}
    ]

    Now evaluate the provided job results and return only the **top 3–5 jobs**.
"""
  matcher_template = PromptTemplate(
    template=template,
    input_variables=["profile_json", "job_results"]
  )
  matcher_chain = matcher_template | llm | parser
  results = matcher_chain.invoke({"profile_json":profile,"job_results":results})
  state["messages"].append(AIMessage(content=results))
  return state

def cover_letter_writer_node(state:MyState):
  parser = StrOutputParser()
  profile = state["messages"][-3].content
  results = state["messages"][-1].content
  template="""
    You are a professional career assistant. Your task is to write personalized cover letters for a candidate based on their profile and job postings.

    Candidate Profile (JSON string):
    {profile_json}

    Top Job Matches (JSON array string, each job has "job_title", "company", "url", "match_score", "reason"):
    {top_jobs}

    Instructions:
    1. Write a **concise, professional cover letter** for each job.
    2. Mention relevant skills, experience, and why the candidate is a good fit.
    3. Keep each letter **unique**, referencing the job title and company.
    4. Output as a JSON array, where each element is:
      {{
          "job_title": "...",
          "company": "...",
          "cover_letter": "..."
      }}

    Example Output:
    [
      {{
        "job_title": "Full Stack Developer",
        "company": "TechCorp",
        "cover_letter": "Dear Hiring Manager, I am excited to apply for Full Stack Developer at TechCorp..."
      }},
      {{
        "job_title": "Backend Engineer",
        "company": "CodeLabs",
        "cover_letter": "Dear Hiring Manager, I am thrilled to apply for Backend Engineer at CodeLabs..."
      }}
    ]

    Now generate cover letters for the candidate based on the top job matches.
"""

  coverletter_template = PromptTemplate(
      template=template,
      input_variables=["profile_json", "top_jobs"]
  )
  chain = coverletter_template | llm | parser
  results = chain.invoke({"profile_json":profile,"top_jobs":results})
  state["messages"].append(AIMessage(content=results))
  return state

### Building the LangGraph Workflow
This section initializes the LangGraph — defining how messages, actions, and model responses connect in the agent’s logic.


In [ ]:
# --------  Langgraph --------

graph = StateGraph(MyState)

graph.add_node("Profile_node",profile_node)
graph.add_node("Job_search_node",job_search_node)
graph.add_node("Matcher_node",matcher_node)
graph.add_node("Cover_letter_writer_node",cover_letter_writer_node)

graph.set_entry_point("Profile_node")

graph.add_edge("Profile_node","Job_search_node")
graph.add_edge("Job_search_node","Matcher_node")
graph.add_edge("Matcher_node","Cover_letter_writer_node")
graph.add_edge("Cover_letter_writer_node",END)

app = graph.compile()

### Creating a Sample Input Message
We prepare an example input to simulate how the Career Agent receives and processes user queries or job-related prompts.




> **Note** : After running the below cell, the output you see is from the `search_jobs_tavily` function. This is to show the intermediate results from the Tavily web search, helping you understand how it is responding and the type of the response.



In [ ]:
input_message = HumanMessage(content="""John Doe
Email: john.doe@example.com | Phone: +1-555-123-4567 | Location: Bangalore, India

Professional Summary:
Highly skilled Full Stack Developer with 5 years of experience in designing, developing, and deploying web applications. Proficient in Python, JavaScript, React, Node.js, and SQL. Strong problem-solving and teamwork skills, with a track record of delivering high-quality software solutions.

Work Experience:
1. Senior Full Stack Developer | TechCorp Solutions | Bangalore, India | Jan 2021 - Present
   - Led a team of 4 developers in building scalable web applications.
   - Developed RESTful APIs using Node.js and integrated with React front-end.
   - Optimized database queries and improved application performance by 30%.

2. Full Stack Developer | CodeLabs Pvt Ltd | Bangalore, India | Jun 2018 - Dec 2020
   - Designed and implemented responsive web interfaces using React and Redux.
   - Collaborated with QA and DevOps teams to ensure smooth deployment.
   - Built automated testing scripts reducing manual testing time by 40%.

Education:
- Bachelor of Technology (B.Tech) in Computer Science | Indian Institute of Technology | 2014 - 2018

Skills:
- Programming: Python, JavaScript, Node.js, React, Redux, SQL, HTML/CSS
- Tools: Git, Docker, AWS, Jenkins, Jira
- Soft Skills: Communication, Teamwork, Problem-solving

Certifications:
- AWS Certified Solutions Architect
- React Developer Certification

Projects:
- E-commerce Platform: Developed a full-stack e-commerce web application with payment integration and real-time inventory management.
- Social Media App: Built a scalable social networking platform supporting thousands of concurrent users.

Languages:
- English (Fluent)
- Hindi (Native)

Interests:
- Open-source contributions, Tech blogging, Chess
""")

final_result  = app.invoke(MyState(messages=[input_message]))
print(type(final_result))

✨ Tavily Search Results: [{'title': 'Senior Full Stack Developer Jobs In Bengaluru Bangalore', 'url': 
'https://www.naukri.com/senior-full-stack-developer-jobs-in-bengaluru-bangalore', 'content': 'Senior Full Stack 
Developer Jobs In Bengaluru Bangalore Image 5 Image 6 Image 37 Image 45 Image 46 Image 47 Image 48 Image 57 Image 
58 Image 59 Image 61: naukri filter iconFilter jobs by Top Companies Image 62: 10476 Apply to 11237 Senior Full 
Stack Developer Jobs In Bengaluru Bangalore on Naukri.com *   •Java Full Stack Developer Jobs In Bangalore *   
•Software Development Engineer Jobs In Bangalore *   •Java Full Stack Developer Jobs In Mumbai *   •Software 
Development Engineer Jobs In Mumbai *   •Software Development Engineer Jobs In Chennai *   •Software Developer Jobs
*   •Java Full Stack Developer Jobs *   •Software Development Engineer Jobs *   senior full stack developer Jobs * 
senior full stack developer Jobs In bengaluru bangalore', 'score': 0.98546}, {'title': '2,000+ Sr.full Stack 
Developer jobs in Bengaluru, Karnataka, India ...', 'url': 
'https://in.linkedin.com/jobs/sr.full-stack-developer-jobs-bengaluru', 'content': 'Get notified about new Sr.full 
Stack Developer jobs in **Bengaluru**. # 2,000+ Sr.full Stack Developer Jobs in Bengaluru ### Software Engineering 
- Full Stack (React + Java) Developer – Manager ### Frontend Developer ### Full Stack Developer ### Backend 
Developer ### Frontend Developer ### Backend Developer ### Full Stack Developer ### Front End Developer ### 
Frontend Developer ### Front end Developer ### Full Stack Developer ### Front End Developer ### Full Stack 
Developer ### Frontend Developer ### Junior Full Stack Developer ### Junior Full stack developer ### Full-stack 
Developer ### Frontend Developer ### Frontend Developer ### Frontend Developer ### Frontend Developer ### Frontend 
Developer ### Frontend Developer ### Full stack developer ### Frontend Developer ### Full Stack Developer ### 
Frontend Developer', 'score': 0.98526}, {'title': 'Senior Full Stack Engineering Tech Lead - 2 to 3 years of 
experience', 'url': 
'https://www.naukri.com/job-listings-senior-full-stack-engineering-tech-lead-shashwath-solution-bengaluru-2-to-3-ye
ars-110925917656', 'content': 'Solid experience with back-end technologies like Node.js, Python, Java, Ruby on 
Rails, or other server-side frameworks. Proficiency in database', 'score': 0.98253}, {'title': '9,000 
Git,jenkins,docker,ansible Job Vacancies in Bengaluru ...', 'url': 
'https://in.indeed.com/Git,jenkins,docker,ansible-jobs-in-Bengaluru,-Karnataka', 'content': 'Work on Frontend, 
Backend, or Full Stack projects. \u2060Build, test, and support web apps. \u2060Debug and document code. Basic 
knowledge: HTML, CSS, JavaScript, React.', 'score': 0.98074}, {'title': 'Senior Full Stack Developer 
(Node.js/JavaScript) in Bengaluru ...', 'url': 
'https://careers.athenahealth.com/us/en/job/R13410/Senior-Full-Stack-Developer-Node-js-JavaScript', 'content': 
'**Position Summary:** We are looking for a\xa0**Full Stack Senior Member of Technical Staff – Software 
Development**\xa0to join our\xa0**athenahealth Insights team**\xa0within the\xa0**Platform division**\xa0at 
our\xa0**Bangalore location**. We offer IT solutions and expert services that eliminate the daily hurdles 
preventing healthcare providers from focusing entirely on their patients — powered by our vision to create a 
thriving ecosystem that delivers accessible, high-quality, and sustainable healthcare for all.* *Along with health 
and financial benefits, athenistas enjoy perks specific to each location, including commuter support, employee 
assistance programs, tuition assistance, employee resource groups, and collaborative*\u202f\u202f*workspaces*\u202f
— *some offices even welcome dogs.* We provide athenistas with a company culture based on learning, the support of 
an engaged team, and an inclusive environment where all employees are valued.*', 'score': 0.97856}, {'title': 
'Senior Full Stac

ℹ️ Type of results: <class 'list'>

ℹ️ Type of results after JSON dump: <class 'str'>

<class 'dict'>

### Observing Results
We execute the full pipeline and observe how the Career Agent responds, showcasing its reasoning and structured output.


In [ ]:
import json

for i, message in enumerate(final_result['messages'], start=1):
    print(f" =================== Message {i} =======================")
    print(f"Type   : {message.type}")
    print("Content:")

    # Try pretty-printing JSON if possible
    try:
        parsed = json.loads(message.content)
        print(json.dumps(parsed, indent=2))
    except:
        print(message.content)


=================== Message 1 =======================

Type   : human

Content:

John Doe
Email: john.doe@example.com | Phone: +1-555-123-4567 | Location: Bangalore, India

Professional Summary:
Highly skilled Full Stack Developer with 5 years of experience in designing, developing, and deploying web 
applications. Proficient in Python, JavaScript, React, Node.js, and SQL. Strong problem-solving and teamwork 
skills, with a track record of delivering high-quality software solutions.

Work Experience:
1. Senior Full Stack Developer | TechCorp Solutions | Bangalore, India | Jan 2021 - Present
   - Led a team of 4 developers in building scalable web applications.
   - Developed RESTful APIs using Node.js and integrated with React front-end.
   - Optimized database queries and improved application performance by 30%.

2. Full Stack Developer | CodeLabs Pvt Ltd | Bangalore, India | Jun 2018 - Dec 2020
   - Designed and implemented responsive web interfaces using React and Redux.
   - Collaborated with QA and DevOps teams to ensure smooth deployment.
   - Built automated testing scripts reducing manual testing time by 40%.

Education:
- Bachelor of Technology (B.Tech) in Computer Science | Indian Institute of Technology | 2014 - 2018

Skills:
- Programming: Python, JavaScript, Node.js, React, Redux, SQL, HTML/CSS
- Tools: Git, Docker, AWS, Jenkins, Jira
- Soft Skills: Communication, Teamwork, Problem-solving

Certifications:
- AWS Certified Solutions Architect
- React Developer Certification

Projects:
- E-commerce Platform: Developed a full-stack e-commerce web application with payment integration and real-time 
inventory management.
- Social Media App: Built a scalable social networking platform supporting thousands of concurrent users.

Languages:
- English (Fluent)
- Hindi (Native)

Interests:
- Open-source contributions, Tech blogging, Chess

=================== Message 2 =======================

...

```json
[
  {
    "job_title": "Senior Full Stack Developer Jobs In Bengaluru Bangalore",
    "company": "Naukri",
    "match_score": 10,
    "reason": "Exact match for current position, location, and highly relevant skills. High score indicates a very 
strong match.",
    "url": "https://www.naukri.com/senior-full-stack-developer-jobs-in-bengaluru-bangalore"
  },
  {
    "job_title": "2,000+ Sr.full Stack Developer jobs in Bengaluru, Karnataka, India ...",
    "company": "LinkedIn",
    "match_score": 10,
    "reason": "Exact match for desired role and location. The content lists relevant technologies like React and 
Java, which are common for full-stack roles.",
    "url": "https://in.linkedin.com/jobs/sr.full-stack-developer-jobs-bengaluru"
  },
  {
    "job_title": "Senior Full Stack Developer (Node.js/JavaScript) in Bengaluru ...",
    "company": "athenahealth",
    "match_score": 9,
    "reason": "Strong match for Senior Full Stack Developer role, location, and key skills like Node.js and 
JavaScript. Experience level is also suitable.",
    "url": "https://careers.athenahealth.com/us/en/job/R13410/Senior-Full-Stack-Developer-Node-js-JavaScript"
  },
  {
    "job_title": "Senior Full-Stack Developer (React, Node.js, Python, Azure)",
    "company": "Quest Global",
    "match_score": 9,
    "reason": "Excellent match for Senior Full Stack Developer role with highly relevant skills (React, Node.js, 
Python). Location is implied to be Bangalore, fitting the profile.",
    "url": 
"https://careers.quest-global.com/global/en/job/P-109320/Senior-Full-Stack-Developer-React-Node-js-Python-Azure"
  },
  {
    "job_title": "Senior Full Stack Software Engineer in Bengaluru, India | Atlassian",
    "company": "Atlassian",
    "match_score": 8,
    "reason": "Good match for Senior Full Stack role and location. Mentions Jira and Bitbucket, aligning with 
candidate's Jira skill. However, it specifies 7+ years of experience, which is slightly more than the candidate's 5
years, but still a strong contender.",
    "url": "https://join.atlassian.com/atlassian-talent-community/jobs/16443?lang=en-us"
  }
]
```

=================== Message 5 =======================

Type   : ai

Content:

```json
[
  {
    "job_title": "Senior Full Stack Developer Jobs In Bengaluru Bangalore",
    "company": "Naukri",
    "cover_letter": "Dear Hiring Manager,\n\nI am writing to express my enthusiastic interest in the Senior Full 
Stack Developer position in Bengaluru, as advertised on Naukri. With 5 years of experience as a Senior Full Stack 
Developer and a strong command of technologies including Python, JavaScript, React, Node.js, SQL, HTML/CSS, Git, 
Docker, and AWS, I am confident in my ability to make a significant contribution to your team.\n\nMy educational 
background, including a B.Tech in Computer Science from the Indian Institute of Technology, coupled with 
certifications like AWS Certified Solutions Architect and React Developer Certification, has provided me with a 
robust foundation in software development principles and best practices. I am adept at problem-solving, teamwork, 
and communication, all of which are crucial for success in a senior development role.\n\nI am particularly drawn to
this opportunity due to its exact match with my current role and location. I am eager to leverage my skills to 
drive innovation and deliver high-quality solutions. Thank you for considering my application. I have attached my 
resume for your review and welcome the opportunity to discuss my qualifications further.\n\nSincerely,\nJohn Doe"
  },
  {
    "job_title": "2,000+ Sr.full Stack Developer jobs in Bengaluru, Karnataka, India ...",
    "company": "LinkedIn",
    "cover_letter": "Dear Hiring Manager,\n\nI am writing with great interest regarding the Sr. Full Stack 
Developer positions available in Bengaluru, Karnataka, India, as listed on LinkedIn. As a Senior Full Stack 
Developer with 5 years of hands-on experience, I possess a comprehensive skill set that aligns perfectly with the 
demands of such roles, including proficiency in Python, JavaScript, React, Node.js, SQL, and cloud technologies 
like AWS.\n\nMy background includes developing and deploying robust web applications, and I am well-versed in 
modern development practices, version control with Git, containerization with Docker, and CI/CD pipelines using 
Jenkins. My educational foundation from IIT and specialized certifications in AWS and React further solidify my 
technical expertise.\n\nI am excited by the prospect of contributing my problem-solving abilities and collaborative
spirit to a forward-thinking organization. I am confident that my proven track record and passion for full-stack 
development make me an ideal candidate. Thank you for your time and consideration. I look forward to hearing from 
you soon.\n\nSincerely,\nJohn Doe"
  },
  {
    "job_title": "Senior Full Stack Developer (Node.js/JavaScript) in Bengaluru ...",
    "company": "athenahealth",
    "cover_letter": "Dear Hiring Manager,\n\nI am writing to express my keen interest in the Senior Full Stack 
Developer position with Node.js/JavaScript in Bengaluru, as advertised by athenahealth. With 5 years of experience 
as a Senior Full Stack Developer, I have a proven track record in building and maintaining complex web 
applications, with a particular emphasis on JavaScript and Node.js.\n\nMy technical expertise includes a strong 
command of JavaScript, Node.js, React, Python, SQL, and cloud platforms like AWS. I am also proficient in Git, 
Docker, and CI/CD tools such as Jenkins. My background, including a B.Tech from IIT and relevant certifications, 
has equipped me with the skills to excel in challenging development environments. I am confident that my ability to
deliver high-quality code, collaborate effectively with teams, and solve intricate technical problems aligns 
perfectly with the requirements of this role.\n\nI am eager to bring my skills and dedication to athenahealth and 
contribute to your innovative projects. Thank you for reviewing my application. I am available for an interview at 
your earliest convenience.\n\nSincerely,\nJohn Doe"
  },
  {
    "job_title": "Senior Full-Sta

# ✅ **Summary & Next Steps**

This notebook demonstrated a full working **Career Agent pipeline**, powered by LLM reasoning and external tools.
Here’s what the agent achieved:

* Parsed a resume into a structured candidate profile
* Generated a job search query and fetched postings via Tavily
* Evaluated job listings based on relevance
* Selected the top matching jobs
* Produced personalized cover letters for each position

All steps were orchestrated using LangGraph’s `StateGraph`, with nodes representing different stages of the workflow.

<br>

## 🔮 **This prepares you for more advanced project notebooks that involve:**

* Exploring alternative orchestration frameworks like LlamaIndex
* Understanding how different LLM tools solve similar problems
* Comparing LangGraph workflows with LlamaIndex pipelines
* Integrating multiple frameworks within one project (optional)
* Evaluating pros/cons of various RAG + agent approaches
<br>

💬 **Tip:**
Try replacing the sample resume with your own to see how the agent adapts the job search and cover letters.